<a href="https://colab.research.google.com/github/lejimmy/terra-python-sdk/blob/main/notebooks/Terra_SDK_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terra SDK for Python - Intro Tutorial
Notebook for William Chen's [YouTube Tutorial](https://youtu.be/GfasBlJHKIg).

In [1]:
# install terra sdk for python
!pip install terra-sdk

# Connect to a Light Client Daemon (LCD) Server
The Terra SDKs rely on an active connection to a running LCD server.

The Light Client Daemon (LCD) provides a REST-based adapter for the RPC endpoints, which also helps for decoding the Amino-encoded blockchain data into parseable JSON. This enables apps to communicate with a node through simple HTTP.

In [2]:
# import the LCDClient module
from terra_sdk.client.lcd import LCDClient

In [3]:
# connect to test net node by specifying the url and chain-id
terra = LCDClient('https://tequila-lcd.terra.dev', 'tequila-0004')

Now that we're connected, we'll try out some available commands.

In [4]:
# node info 
terra.tendermint.node_info()

{'application_version': {'build_deps': ['github.com/99designs/keyring@v1.1.6',
   'github.com/ChainSafe/go-schnorrkel@v0.0.0-20200405005733-88cbf1b4c40d',
   'github.com/CosmWasm/go-cosmwasm@v0.10.0 => github.com/terra-project/go-cosmwasm@v0.10.4',
   'github.com/Workiva/go-datastructures@v1.0.52',
   'github.com/bartekn/go-bip39@v0.0.0-20171116152956-a05967ea095d',
   'github.com/beorn7/perks@v1.0.1',
   'github.com/bgentry/speakeasy@v0.1.0',
   'github.com/btcsuite/btcd@v0.20.1-beta',
   'github.com/btcsuite/btcutil@v1.0.2',
   'github.com/cespare/xxhash/v2@v2.1.1',
   'github.com/cosmos/cosmos-sdk@v0.39.2 => github.com/terra-project/cosmos-sdk@v0.39.2-public.4',
   'github.com/cosmos/go-bip39@v0.0.0-20180819234021-555e2067c45d',
   'github.com/cosmos/ledger-cosmos-go@v0.11.1 => github.com/terra-project/ledger-terra-go@v0.11.1-terra',
   'github.com/cosmos/ledger-go@v0.9.2',
   'github.com/davecgh/go-spew@v1.1.1',
   'github.com/dvsekhvalnov/jose2go@v0.0.0-20200901110807-248326c1351b

In [5]:
# oracle parameters
terra.oracle.parameters()

{'min_valid_per_window': '0.050000000000000000',
 'reward_band': '0.020000000000000000',
 'reward_distribution_window': '5256000',
 'slash_fraction': '0.000100000000000000',
 'slash_window': '100800',
 'vote_period': '5',
 'vote_threshold': '0.500000000000000000',
 'whitelist': [{'name': 'ukrw', 'tobin_tax': '0.010000000000000000'},
  {'name': 'usdr', 'tobin_tax': '0.003500000000000000'},
  {'name': 'uusd', 'tobin_tax': '0.003500000000000000'},
  {'name': 'umnt', 'tobin_tax': '0.020000000000000000'},
  {'name': 'ueur', 'tobin_tax': '0.003500000000000000'},
  {'name': 'ucny', 'tobin_tax': '0.003500000000000000'},
  {'name': 'ujpy', 'tobin_tax': '0.003500000000000000'},
  {'name': 'ugbp', 'tobin_tax': '0.003500000000000000'},
  {'name': 'uinr', 'tobin_tax': '0.003500000000000000'},
  {'name': 'ucad', 'tobin_tax': '0.003500000000000000'},
  {'name': 'uchf', 'tobin_tax': '0.003500000000000000'},
  {'name': 'uhkd', 'tobin_tax': '0.003500000000000000'},
  {'name': 'usgd', 'tobin_tax': '0.003

In [6]:
# block height of last query
terra.last_request_height

'5175506'

In [7]:
# current tax rate
terra.treasury.tax_rate()

Dec('0.002467377735563951')

# Create a New Wallet

In [8]:
from terra_sdk.key.mnemonic import MnemonicKey

# recover an existing wallet
mk = MnemonicKey(mnemonic='secret globe hint gather move evil monster column disorder burden couple odor salad holiday small float endorse alley escape veteran cruel cross web during')

In [9]:
# wallet address
mk.acc_address

'terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx'

In [10]:
# private key in bytes
mk.private_key

b"\xef\xbd8d3\x81Y\xfb\xbdM\x89 \x81\xe2Z\xf6'`\xed\xe8!\xed\xd4svy8\xc6\x12\x03\x9c\x9c"

In [11]:
# create a second wallet
mk2 = MnemonicKey()

In [12]:
# check wallet balance
terra.bank.balance(mk.acc_address)

Coins('499425343uluna,2497574910uusd')

In [13]:
# check wallet denominations
terra.bank.balance(mk.acc_address).denoms()

['uluna', 'uusd']

In [14]:
# wallet balance of luna
terra.bank.balance(mk.acc_address)['uluna']

Coin(denom='uluna', amount=499425343)

In [15]:
# wallet value of luna
terra.bank.balance(mk.acc_address)['uluna'].amount

499425343

## Sending Transactions

To transfer between native Terra assets, use `MsgSend` from the `Bank` module.

In [16]:
# import `MsgSend` module
from terra_sdk.core.bank import MsgSend

In [17]:
# create a message sending half of all coins
send = MsgSend(mk.acc_address, mk2.acc_address, terra.bank.balance(mk.acc_address)/2)
send

MsgSend(from_address='terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx', to_address='terra1x5tcalqz444k6ahjjv0nkkc3pk9sv6fzfgscp4', amount=Coins('249712671uluna,1248787455uusd'))

In [18]:
# create a wallet from the mnemonic key
wallet = terra.wallet(mk)

In [19]:
# create and sign a send msg
tx = wallet.create_and_sign_tx(
    msgs=[send],
    memo='Hello There!',
    gas_prices='2uluna',
    gas_adjustment='1.5',
    fee_denoms=['uluna', 'uusd']
    )
tx

StdTx(msg=[MsgSend(from_address='terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx', to_address='terra1x5tcalqz444k6ahjjv0nkkc3pk9sv6fzfgscp4', amount=Coins('249712671uluna,1248787455uusd'))], fee=StdFee(gas=107536, amount=Coins('215072uluna,1425090uusd')), signatures=[StdSignature(signature='dnPsetRRb4YQI0gGidfxSomnYEz0FeAjwfsXxsrEIfYWHbNwKdhC1oVojzrZEBJVSKwwfwOjtamz/p8zK+Tt9A==', pub_key=PublicKey(type='tendermint/PubKeySecp256k1', value='A1Qh3vrRBgq46YITAxUX1oI9SzAKUrmLidbIxWmOSea0'))], memo='Hello There!')

In [20]:
# broadcast transaction
result = terra.tx.broadcast(tx)

In [21]:
# result of broadcast
result

BlockTxBroadcastResult(height=5175513, txhash='77B9BAA84B43DFA5BF136BCD672E5B14C9FC4D725BB7F635BEDFD235F6B7529D', raw_log='[{"msg_index":0,"log":"","events":[{"type":"message","attributes":[{"key":"action","value":"send"},{"key":"sender","value":"terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx"},{"key":"module","value":"bank"}]},{"type":"transfer","attributes":[{"key":"recipient","value":"terra1x5tcalqz444k6ahjjv0nkkc3pk9sv6fzfgscp4"},{"key":"sender","value":"terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx"},{"key":"amount","value":"249712671uluna,1248787455uusd"}]}]}]', gas_wanted=107536, gas_used=78723, logs=[TxLog(msg_index=0, log='', events=[{'type': 'message', 'attributes': [{'key': 'action', 'value': 'send'}, {'key': 'sender', 'value': 'terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx'}, {'key': 'module', 'value': 'bank'}]}, {'type': 'transfer', 'attributes': [{'key': 'recipient', 'value': 'terra1x5tcalqz444k6ahjjv0nkkc3pk9sv6fzfgscp4'}, {'key': 'sender', 'value': 'terra1la8v6shy3gaumtqwj9

In [22]:
 # result transaction hash
 result.txhash

'77B9BAA84B43DFA5BF136BCD672E5B14C9FC4D725BB7F635BEDFD235F6B7529D'

In [23]:
# click here to view transaction
print(f'https://finder.terra.money/tequila-0004/tx/{result.txhash}')

https://finder.terra.money/tequila-0004/tx/77B9BAA84B43DFA5BF136BCD672E5B14C9FC4D725BB7F635BEDFD235F6B7529D


# Swap
To swap between two coins, use the `MsgSwap` module.

In [24]:
# import the MsgSwap module
from terra_sdk.core.market import MsgSwap

In [25]:
# message swap of 1 UST for luna
swap = MsgSwap(mk.acc_address, '1000000uusd', 'uluna')

In [26]:
# create and sign swap message
tx = wallet.create_and_sign_tx(
    msgs=[swap],
    gas_prices='2uluna',
    gas_adjustment='1.5',
    fee_denoms=['uluna', 'uusd']
    )
tx

StdTx(msg=[MsgSwap(trader='terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx', offer_coin=Coin(denom='uusd', amount=1000000), ask_denom='uluna')], fee=StdFee(gas=216454, amount=Coins('432908uluna')), signatures=[StdSignature(signature='7uyX1kMNl1xGChcPh0qG7WrCNGpq9/FXjtOok+vG4PsmnXXDlXPqjr1a2sq0IvtiT8JOS7KIFpEMkV6tiC/AVg==', pub_key=PublicKey(type='tendermint/PubKeySecp256k1', value='A1Qh3vrRBgq46YITAxUX1oI9SzAKUrmLidbIxWmOSea0'))], memo='')

In [27]:
# broadcast transaction
result = terra.tx.broadcast(tx)
result

BlockTxBroadcastResult(height=5175517, txhash='764256A6C018260FC3F4D08ECD927C178AF5680AA8C757E4FC9AEF34E9D6C2AF', raw_log='[{"msg_index":0,"log":"","events":[{"type":"message","attributes":[{"key":"action","value":"swap"},{"key":"sender","value":"terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx"},{"key":"sender","value":"terra1untf85jwv3kt0puyyc39myxjvplagr3wstgs5s"},{"key":"module","value":"market"}]},{"type":"swap","attributes":[{"key":"offer","value":"1000000uusd"},{"key":"trader","value":"terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx"},{"key":"recipient","value":"terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx"},{"key":"swap_coin","value":"84622uluna"},{"key":"swap_fee","value":"426.167311754411850275uluna"}]},{"type":"transfer","attributes":[{"key":"recipient","value":"terra1untf85jwv3kt0puyyc39myxjvplagr3wstgs5s"},{"key":"sender","value":"terra1la8v6shy3gaumtqwj94g54nsedpa26yxmfdfzx"},{"key":"amount","value":"1000000uusd"},{"key":"recipient","value":"terra1la8v6shy3gaumtqwj94g54nsedpa26y

In [28]:
# click here to view transaction
print(f'https://finder.terra.money/tequila-0004/tx/{result.txhash}')

https://finder.terra.money/tequila-0004/tx/764256A6C018260FC3F4D08ECD927C178AF5680AA8C757E4FC9AEF34E9D6C2AF
